In [1]:
# read table data
from pygmid import Lookup as lk
import numpy as np
nmos = lk('./simulation/sg13_hv_nmos.mat')
pmos = lk('./simulation/sg13_hv_pmos.mat')
# list of parameters: VGS, VDS, VSB, L, W, NFING, ID, VT, GM, GMB, GDS, CGG, CGB, CGD, CGS, CDD, CSS, STH, SFL
# if not specified, minimum L, VDS=max(vgs)/2=0.9 and VSB=0 are used 

In [2]:
# define the given parameters as taken from the specification table or inital guesses
c_load = 0.1e-12
gm_id_m12 = 15
gm_id_m34 = 10
gm_id_m56 = 8
l_12 = 3
l_34 = 3
l_56 = 3
f_bw = 10e6 # -3dB bandwidth of the voltage buffer
i_total_limit = 8e-6
i_bias_in = 8e-6
output_voltage = 3
vin_min = 0.1
vin_max = 2.9
vdd_min = 3
vdd_max = 3

In [3]:
# we get the required gm of M1/2 from the -3dB bandwidth requirement of the voltage buffer specification
# note that the -3dB bandwidth of the voltage buffer with gain Av=1 is equal to the unity gain bandwidth
# of the ota, hence we wet them equal here
# we add a factor of 3 to allow for PVT variation plus additional MOSFET parasitic loading
gm_m12 = f_bw * 3 * 4*np.pi*c_load
print('gm12 =', round(gm_m12/1e-3, 4), 'mS')

gm12 = 0.0377 mS


In [4]:
# since we know gm12 and the gmid we can calculate the bias current
id_m12 = gm_m12 / gm_id_m12
i_total = 2*id_m12
print('i_total (exact) =', round(i_total/1e-6, 1), 'µA')
# we round to 0.5µA bias currents
i_total = max(round(i_total / 1e-6 * 2) / 2 * 1e-6, 0.5e-6)
id_m12 = i_total/2

print('i_total (rounded) =', i_total/1e-6, 'µA')
if i_total < i_total_limit:
    print('[info] power consumption target is met!')
else:
    print('[info] power consumption target is NOT met!') 

i_total (exact) = 5.0 µA
i_total (rounded) = 5.0 µA
[info] power consumption target is met!


In [5]:
# we calculate the dc gain
gm_gds_m12 = nmos.lookup('GM_GDS', GM_ID=gm_id_m12, L=l_12, VDS=1.65, VSB=0)
gm_gds_m34 = pmos.lookup('GM_GDS', GM_ID=gm_id_m34, L=l_34, VDS=1.65, VSB=0)

gds_m12 = gm_m12 / gm_gds_m12
gm_m34 = gm_id_m34 * i_total/2
gds_m34 = gm_m34 / gm_gds_m34

a0 = gm_m12 / (gds_m12 + gds_m34)
print('a0 =', round(20*np.log10(a0), 1), 'dB')

a0 = 52.4 dB


In [6]:
# we can now look up the VGS of the MOSFET
vgs_m12 = nmos.look_upVGS(GM_ID=gm_id_m12, L=l_12, VDS=1.65, VSB=0.0)
vgs_m34 = pmos.look_upVGS(GM_ID=gm_id_m34, L=l_34, VDS=1.65, VSB=0.0) 
vgs_m56 = nmos.look_upVGS(GM_ID=gm_id_m56, L=l_56, VDS=1.65, VSB=0.0) 

print('vgs_12 =', round(float(vgs_m12), 3), 'V')
print('vgs_34 =', round(float(vgs_m34), 3), 'V')
print('vgs_56 =', round(float(vgs_m56), 3), 'V')

vgs_12 = 0.64 V
vgs_34 = 0.805 V
vgs_56 = 0.802 V


In [7]:
# calculate all widths
id_w_m12 = nmos.lookup('ID_W', GM_ID=gm_id_m12, L=l_12, VDS=vgs_m12, VSB=0)
w_12 = id_m12 / id_w_m12
w_12_round = max(round(w_12*2)/2, 0.5)
print('M1/2 W =', round(w_12, 2), 'um, rounded W =', w_12_round, 'um')

id_m34 = id_m12
id_w_m34 = pmos.lookup('ID_W', GM_ID=gm_id_m34, L=l_34, VDS=vgs_m34, VSB=0)
w_34 = id_m34 / id_w_m34
w_34_round = max(round(w_34*2)/2, 0.5) 
print('M3/4 W =', round(w_34, 2), 'um, rounded W =', w_34_round, 'um')

id_w_m5 = nmos.lookup('ID_W', GM_ID=gm_id_m56, L=l_56, VDS=vgs_m56, VSB=0)
w_5 = i_total / id_w_m5
w_5_round = max(round(w_5*2)/2, 0.5)
print('M5 W =', round(w_5, 2), 'um, rounded W =', w_5_round, 'um')
w_6 = w_5_round * i_bias_in / i_total
w_6_round = max(round(w_6*2)/2, 0.5)
print('M6 W =', round(w_6_round, 2), 'um')

M1/2 W = 19.31 um, rounded W = 19.5 um
M3/4 W = 18.31 um, rounded W = 18.5 um
M5 W = 6.5 um, rounded W = 6.5 um
M6 W = 10.5 um


In [8]:
# print out final design values
print('5T-OTA dimensioning:')
print('--------------------')
print('M1/2 W=', w_12_round, ', L=', l_12)
print('M3/4 W=', w_34_round, ', L=', l_34)
print('M5   W=', w_5_round, ', L=', l_56)
print('M6   W=', w_6_round, ', L=', l_56)
print()
print('5T-OTA performance summary:')
print('---------------------------')
print('supply current =', round(i_total/1e-6, 1), 'µA')

5T-OTA dimensioning:
--------------------
M1/2 W= 19.5 , L= 3
M3/4 W= 18.5 , L= 3
M5   W= 6.5 , L= 3
M6   W= 10.5 , L= 3

5T-OTA performance summary:
---------------------------
supply current = 5.0 µA
